This notebook will ensure that you can run the measurement code, and moreover, it will outline what the different components of the notebook accomplish. The purpose of each section of the notebook should be clear and users should be able to create real measurements after creating their own subclasses for their specific setup. 

In [ ]:
from dreye.hardware import dummy_system
from dreye.hardware import dummy_spectrometer
from dreye.core.spectrum_utils import create_gaussian_spectrum
import numpy as np
import matplotlib.pyplot as plt
import warnings

First, specify the wavelength range you are interested in (in this case we used a range of 100 to 1000 nm, with steps of 0.1nm). Then, create dummy LEDS using the **create_gaussian_spectrum** class. We chose to create 6. You can plot these spectra for visualizaiton purposes. 

In [ ]:
wls = np.arange(100, 1000, 0.1)
dummy_leds = create_gaussian_spectrum(
    wls, 
    [340, 360, 415, 455, 565, 615], 
    std=8, 
    units='spectralirradiance'
)
plt.plot(dummy_leds.domain, dummy_leds)
plt.xlabel('wavelengths (nm)')
plt.ylabel('irradiance')

Next, create dummy voltage analog outputs (AOs). 
 - Specify the names of the LEDS (our examples range from dUV to orange)
 - Specify the channel names as a list of strings. The channel names correspond to the method by which the LEDs in the system are controlled- in this case we are using the National Instruments subclass for the sake of testing, but subclasses exist for other devices (e.g. Arduino). 
 - Specify zero boundaries: the value of each LED which results in a 0 intensity 
 - Specify the max boundariers: the max value possible for each LED. * * Note: you can change units to time, if you are using alternate intensity methods such as pulse width modulation. <br><br> Overall, this cell creates list of dummy AOs and puts them all into one system.

In [ ]:
leds = ['duv', 'uv', 'violet', 'rblue', 'lime', 'orange']
channels = ['Dev2/ao1', 'Dev3/ao1', 
            'Dev3/ao0', 'Dev1/ao1', 
            'Dev2/ao0', 'Dev1/ao0']
zeros = [4.4] * 5 + [0.]
maxs = [0.] * 5 + [5.]

outputs = []
for led, ch, z, m in zip(leds, channels, zeros, maxs):
    outputs.append(
        dummy_system.DummyOutput(
            ch, led, 
            zero_boundary=z, 
            max_boundary=m, 
            units='V'
        )
    )

Next, create **system**, which is a method to control all of the LEDs simultaneously.

In [ ]:
system = dummy_system.DummySystem(outputs)

You can see the details of each dummy output below:

In [ ]:
system

Next, create a dummy spectrophotometer. Specify the desired level of artificial noise. 

In [ ]:
spec = dummy_spectrometer.Spectrometer(
    wls, 
    dummy_leds, 
    system,
    noise_scale=2, 
)

**runner** is a class that you pass your spectrophotometer and system instance to. 
 - wls: If set to none, wavelengths measured will be set to whatever is spit out by the spectrophotometer. Alternatively, provide a numpy array with a specific numpy array (see comment). 
 - smoothing window: boxcar smoothing window to smooth the spectrum of each avereraged intensity value of each LED
 - n_steps: the number of steps from 0 boundary to max boundary per LED, inclusive
 - n_avg: the number of times each step is averaged over
 - remove_zero: substracts the zero boundary from all the other measurements. if you dont want to remove it, set it to false. 

We entered arbitrary numbers here, but we reccomend starting with the default values when running an actual measurement. Therefore, initially specifying keywords may be unecessary. Tweak the parameters as needed. 

In [ ]:
from dreye.hardware.measurement_runner import MeasurementRunner

In [ ]:
runner = MeasurementRunner(
    system, 
    spec, 
    wls=None, # np.arange(200, 800, 1), 
    smoothing_window=1, 
    n_steps=10, 
    n_avg=1, 
    remove_zero=False, 
)

You're now ready to run the measurement. Set the verbosity level: **verbose**=0 will display nothing, verbose=1 will display minimal updates, verbose=2 will display both voltages for the LED measurements and photons counted per second.

In [ ]:
runner.run(verbose=2)

After the measurement, the system will have a new attribute called **spms**- standing for spectral measurement. This attribute is its own class containign wavelength values across intensities, and is used for fitting photoreceptors. You can also plot **spms** to visualize your measurements. 

In [ ]:
system.spms.units

In [ ]:
plt.plot(
    system.spms.domain, system.spms
)
plt.xlabel('Volts (V)')
plt.ylabel('photon flux')

You can simply map the measured intensity values to voltage output that you have to give. In this example, intensities are 30 microE_Q, and we are asking what voltage value must be applied to each LED to reach this intensity. 

In [ ]:
system.spms.map(np.array([30000]*6))

**spms** stores both the measured spectrum (intensity across volts) and the normalized spectrum plotted below, where each LED spectrum is normalized with as an integral of 1

In [ ]:
plt.plot(
    system.spms.normalized_spectrum.wavelengths, 
    system.spms.normalized_spectrum)
plt.xlabel('wavelengths')
plt.ylabel('normalized flux intensity')

Save the system instance that you have created (including the measurement (**spms**) and hardware settings)

In [ ]:
runner.save('measurement_test.json')

Finally, you can reload the system to plot measurements, etc

In [ ]:
from dreye.io import read_json

In [ ]:
system_loaded = read_json('measurement_test.json')

In [ ]:
print(system_loaded)

In [ ]:
plt.plot(
    system_loaded.spms.domain, system_loaded.spms
)
plt.xlabel('Volts (V)')
plt.ylabel('photon flux')

In [ ]:
plt.plot(
    system_loaded.spms.normalized_spectrum.wavelengths, 
    system_loaded.spms.normalized_spectrum)
plt.xlabel('wavelengths')
plt.ylabel('normalized flux intensity')

After confirming that the code runs on your system, it's time to create your real measurements. Make sure that the subclasses you use are appropriate for your LED system and spectrophotometer. 